In [1]:
# import shutil 
# import urllib.request as request 
# from contextlib import closing 

# with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
#     with open('sift.tar.gz', 'wb') as f:
#         shutil.copyfileobj(r, f)

In [2]:
# import tarfile 

# tar = tarfile.open('Data/sift.tar.gz', 'r:gz')
# tar.extractall()

In [3]:
import numpy as np 

def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [4]:
xb = read_fvecs('sift/sift_base.fvecs')
xq = read_fvecs('sift/sift_query.fvecs')

In [5]:
xb.shape, xq.shape 

((1000000, 128), (10000, 128))

In [6]:
xq = xq[0].reshape(1, xq.shape[1])
xq.shape

(1, 128)

In [7]:
xq

array([[  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
         18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
        101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
          1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
         40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
        116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
         10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
          0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
        116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
          9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
          2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
         18.,   8.,   2.,  19.,  42.,  48.,  11.]], dtype=float32)

In [8]:
import faiss

d = 128 
k = 10 

index = faiss.IndexFlatIP(d)
index.add(xb)
D, I1 = index.search(xq, k)
I1[0]

array([932085, 934876, 561813, 708177, 706771, 695756, 435345, 701258,
       872728, 455537], dtype=int64)

In [9]:
nbits = d * 4 

index = faiss.IndexLSH(d, nbits)
index.add(xb)
D, I2 = index.search(xq, k)
I2[0]

array([435345, 931632, 708177, 813701, 934876, 455537, 932085, 561813,
       248185, 361496], dtype=int64)

In [10]:
np.in1d(I1[0], I2[0])

array([ True,  True,  True,  True, False, False,  True, False, False,
        True])

In [11]:
M = 64 
ef_search = 32 
ef_construction = 64 

index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efConstruction = ef_construction 
index.hnsw.efSearch = ef_search 
index.add(xb)

D, I = index.search(xq, k)
I[0]

array([932085, 934876, 561813, 708177, 706771, 695756, 435345, 701258,
       455537, 872728], dtype=int64)

In [12]:
nlist = 128 

quantizer = faiss.IndexFlatIP(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)
index.train(xb)
index.add(xb)
index.nprobe = 8 

D, I = index.search(xq, k)
I[0]

array([932085, 934876, 561813, 708177, 706771, 695756, 435345, 701258,
       455537, 872728], dtype=int64)